In [1]:
import copy
import numpy as np
from sympy.combinatorics import Permutation
Permutation.print_cyclic = False

In [2]:
EPS=1e-9

In [3]:
n=int(input("poryadok sistemi "))

poryadok sistemi 3


In [66]:
matrix=[]
for i in range(n):
    matrix.append(list(map(float,input().split())))
matrix=np.array(matrix)

1 2 3
4 8 1
2 3 5


In [ ]:
'''
9.331343 1.120045 -2.880925 
1.120045 7.086042 0.670297 
-2.880925 0.670297 5.622534 
'''

'''
6.687233 0.80267 -20.6459 0
0.80267 5.07816 0.48037 1
-2.06459 0.48037 4.02934 0


'''

In [33]:
def create_system(a,b):
    a=np.array(a,dtype="float64")
    b=np.array(b,dtype="float64").reshape(-1,1)
    return np.concatenate((a,b),axis=1)

def gauss_method(matrix,n=n,EPS=EPS,choice=False):
    #matrix.dtype=np.float
    perm_list=[]
    matrix=matrix.astype("float64")
    m=len(matrix[0])
    for i in range(n):
        if (not choice):
            tmp=matrix[i][i]
            if abs(tmp)<EPS:
                print("Vozmozhno delenie na malenkoe chislo")
                for j in range(i+1,n):
                    if (matrix[j][i])>EPS:
                        matrix[[i,j]]=matrix[[j,i]]
                    elif (j==n-1):
                        i+=1
        else:
            sort_a=matrix[0:n,i:n]
            perm=sort_a[i].argsort()[::-1]
            sort_a=sort_a[:,perm]
            perm+=i
            perm=np.concatenate((np.array(range(n-len(perm))),perm))
            perm_list.append(perm)
            matrix[0:n,i:n]=sort_a
            tmp=matrix[i][i]
        for j in range(i,m):
            matrix[i][j]/=tmp
        for j in range(i+1,n):
            matrix[j]-=(matrix[i]*matrix[j][i])        
    for i in range(n-1,-1,-1):
        if (matrix[i][i]):
            for j in range(i-1,-1,-1):
                matrix[j]-=(matrix[i]*matrix[j][i])
    if choice:
        return matrix,perm_list
    else:
        return matrix,None

In [65]:
iden=np.identity(n)
matrix_copy=copy.deepcopy(matrix)
invert=np.concatenate((matrix_copy,iden),axis=1)
print(invert)

[[  6.687233   0.80267  -20.6459     1.         0.         0.      ]
 [  0.80267    5.07816    0.48037    0.         1.         0.      ]
 [ -2.06459    0.48037    4.02934    0.         0.         1.      ]]


In [60]:
invert,tr=gauss_method(invert)

In [61]:
invert[0:n,n:2*n]

array([[-0.2186595 ,  0.14214846, -1.13733414],
       [ 0.04567545,  0.16947465,  0.21383161],
       [-0.11748409,  0.05263089, -0.36007063]])

In [62]:
condition_number=np.linalg.norm(invert[0:n,n:2*n],"fro")*np.linalg.norm(matrix,"fro")
print(condition_number)

28.675178320882495


In [63]:
matrix_copy.dot(invert[0:n,n:2*n])

array([[ 1.00000000e+00,  9.09400631e-17, -5.03902861e-16],
       [ 6.92972929e-18,  1.00000000e+00,  1.11555452e-16],
       [ 7.09041529e-17, -3.22017085e-17,  1.00000000e+00]])

In [67]:
b=np.array(list(map(float,input("pravaya chast' ").split()))).reshape((-1,1))

pravaya chast' 17 24 28


In [64]:
#7.570463 8.876384 3.411906
# 0 1 0

In [68]:
equations=copy.deepcopy(matrix)
#equations=np.concatenate((equations,b),axis=1)
equations=create_system(equations,b)
print(equations)
equations,perm_list=gauss_method(equations)
print(equations)
if (perm_list is  None):
    answer=equations[0:n,-1]
else:
    id_p=Permutation(range (n))
    for perm in perm_list:
        id_p=id_p*Permutation(perm)
    answer=(equations[0:n,-1])[((id_p).array_form)]
print()
print(answer)
print()
print(matrix@(answer.reshape((-1,1)))-b)

[[ 1.  2.  3. 17.]
 [ 4.  8.  1. 24.]
 [ 2.  3.  5. 28.]]
Vozmozhno delenie na malenkoe chislo
[[  1.  60.  nan  nan]
 [  0. -11.  nan  nan]
 [  0.  -1.  nan  nan]]

[nan nan nan]

[[nan]
 [nan]
 [nan]]


/tmp/ipykernel_3736/1434128690.py:33: RuntimeWarning: divide by zero encountered in double_scalars
  matrix[i][j]/=tmp
/tmp/ipykernel_3736/1434128690.py:33: RuntimeWarning: invalid value encountered in double_scalars
  matrix[i][j]/=tmp


In [69]:
equations=copy.deepcopy(matrix)
#equations=np.concatenate((equations,b),axis=1)
equations=create_system(equations,b)
equations,perm_list=gauss_method(equations,choice=True)
print(equations)
if (perm_list is  None):
    answer=equations[0:n,-1]
else:
    id_p=Permutation(range (n))
    for perm in perm_list:
        id_p=id_p*Permutation(perm)
    answer=(equations[0:n,-1])[((id_p).array_form)]
print()
print(answer)
print()
print(matrix@(answer.reshape((-1,1)))-b)

[[1. 0. 0. 4.]
 [0. 1. 0. 2.]
 [0. 0. 1. 1.]]

[1. 2. 4.]

[[ 0.00000000e+00]
 [ 0.00000000e+00]
 [-3.55271368e-15]]


In [46]:
def LU_decomposition(matrix,b,n):
    matrix=np.array(matrix)

    b=np.array(b,dtype="float64").reshape((-1,1))
    matrix=np.concatenate((matrix,b),axis=1)
    L=np.zeros((n,n),dtype="float64")
    U=np.identity(n,dtype="float64")
    U=np.concatenate((U,b),axis=1)
    for i in range(n):
        for j in range(i,n):
            s=0
            for k in range(i):
                s+=L[j][k]*U[k][i]
            L[j][i]=matrix[j][i]-s
        for j in range (i,n+1):
            s=0
            for k in range (i):
                s+=L[i][k]*U[k][j]
            U[i][j]=(matrix[i][j]-s)/L[i][i]
    return L,U
            
    

In [47]:
matrix_d=copy.deepcopy(matrix)
L,U=LU_decomposition(matrix_d,b,n)

In [48]:
L

array([[ 6.687233  ,  0.        ,  0.        ],
       [ 0.80267   ,  4.98181535,  0.        ],
       [-2.06459   ,  0.72818318, -2.77723291]])

In [49]:
U

array([[ 1.        ,  0.12003021, -3.08736065,  0.        ],
       [ 0.        ,  1.        ,  0.59386018,  0.20073004],
       [ 0.        ,  0.        ,  1.        ,  0.05263089]])

In [50]:
L.dot(U)

array([[ 6.6872330e+00,  8.0267000e-01, -2.0645900e+01,  0.0000000e+00],
       [ 8.0267000e-01,  5.0781600e+00,  4.8037000e-01,  1.0000000e+00],
       [-2.0645900e+00,  4.8037000e-01,  4.0293400e+00,  1.6449154e-20]])

In [51]:
print(matrix,"\n",b)

[[  6.687233   0.80267  -20.6459  ]
 [  0.80267    5.07816    0.48037 ]
 [ -2.06459    0.48037    4.02934 ]] 
 [[0.]
 [1.]
 [0.]]


In [52]:
y=U[0:n,-1]
print(y)

[0.         0.20073004 0.05263089]


In [53]:

lu_ans,tr=gauss_method(U,n)
print(lu_ans)
print()
print(lu_ans[0:n,-1])
print()
print(matrix@(lu_ans[0:n,-1].reshape((-1,1)))-b)

[[1.         0.         0.         0.14214846]
 [0.         1.         0.         0.16947465]
 [0.         0.         1.         0.05263089]]

[0.14214846 0.16947465 0.05263089]

[[ 9.09400631e-17]
 [ 0.00000000e+00]
 [-3.22017085e-17]]


In [54]:
lu_ans

array([[1.        , 0.        , 0.        , 0.14214846],
       [0.        , 1.        , 0.        , 0.16947465],
       [0.        , 0.        , 1.        , 0.05263089]])

In [55]:
matrix_c=copy.deepcopy(matrix)

In [56]:
matrix_c[0][0]=1e-10*matrix[0][0]

In [57]:
equations=copy.deepcopy(matrix_c)
#equations=np.concatenate((equations,b),axis=1)
equations=create_system(equations,b)
equations,perm_list=gauss_method(equations)
print(equations)
if (perm_list is  None):
    answer=equations[0:n,-1]
else:
    id_p=Permutation(range (n))
    for perm in perm_list:
        id_p=id_p*Permutation(perm)
    answer=(equations[0:n,-1])[((id_p).array_form)]
print()
print(answer)
print()
print(matrix_c@(answer.reshape((-1,1)))-b)

Vozmozhno delenie na malenkoe chislo
[[220.91839946   0.           0.           1.        ]
 [-43.18674141   1.           0.           0.        ]
 [ -1.67901141   0.           1.           0.        ]]

[1. 0. 0.]

[[ 6.687233e-10]
 [-1.973300e-01]
 [-2.064590e+00]]


In [58]:
equations=copy.deepcopy(matrix_c)
#equations=np.concatenate((equations,b),axis=1)
equations=create_system(equations,b)
equations,perm_list=gauss_method(equations,choice=True)
print(equations)
if (perm_list is  None):
    answer=equations[0:n,-1]
else:
    id_p=Permutation(range (n))
    for perm in perm_list:
        id_p=id_p*Permutation(perm)
    answer=(equations[0:n,-1])[((id_p).array_form)]
print()
print(answer)
print()
print(matrix_c@(answer.reshape((-1,1)))-b)

[[1.         0.         0.         0.18710835]
 [0.         1.         0.         0.00727439]
 [0.         0.         1.         0.05773166]]

[0.05773166 0.18710835 0.00727439]

[[ 2.38447370e-18]
 [ 2.22044605e-16]
 [-2.31715822e-17]]
